In [1]:
import numpy as np
import pandas as pd
import pulp
from amply import Amply

<img src="./images/example_lp_formulation.jpg" width=500 height=500 align="left"/>

#### Objective Function
#### Maximize Profit
#### $30 X_{Desks} + 16 X_{Chairs} + 25 X_{Bookcases}$

In [2]:
data_str = """
set PRODUCTS := Desks Chairs Bookcases;
set OPERATIONS := Cutting Finishing Storing;

set PRODUCT_OPERATIONS dimen 2;
set PRODUCT_OPERATIONS : Desks Chairs Bookcases :=
 Cutting      + + +
 Finishing    + + +
 Storing      + + +
;

param OPERATION_REQUIREMENTS{PRODUCT_OPERATIONS};
param OPERATION_REQUIREMENTS :=
 Cutting Desks 0.8
 Cutting Chairs 0.4
 Cutting Bookcases 0.4
 Finishing Desks 5
 Finishing Chairs 3
 Finishing Bookcases 3
 Storing Desks 1
 Storing Chairs 0.7
 Storing Bookcases 1.1
;

param CAPACITY_AVAILABLE{OPERATIONS};
param CAPACITY_AVAILABLE :=
 Cutting 100
 Finishing 650
 Storing 140
;

param PROFIT{PRODUCTS};
param PROFIT :=
 Desks 30
 Chairs 16
 Bookcases 25
;
"""

In [3]:
data = Amply(data_str)

In [4]:
data.PRODUCTS

<SetObject: ['Desks', 'Chairs', 'Bookcases']>

### Define Problem

In [5]:
prob = pulp.LpProblem("ProductMix",pulp.LpMaximize)

### Decision Variables (DVs)

In [6]:
product_dvs = {}
for product in data.PRODUCTS:
    var = pulp.LpVariable(name=product,lowBound=0)
    product_dvs[product] = var

### Objective Function

In [7]:
obj_expression = pulp.LpAffineExpression()
for product in data.PRODUCTS:
    obj_expression += product_dvs[product] * data.PROFIT[product]

In [8]:
prob += obj_expression

### Constraints

In [9]:
constraints = []
for operation in data.OPERATIONS:
    expression = pulp.LpAffineExpression()
    for product in data.PRODUCTS:
        if (operation,product) in data.PRODUCT_OPERATIONS:
            expression += product_dvs[product] * data.OPERATION_REQUIREMENTS[(operation,product)]
    expression = expression <= data.CAPACITY_AVAILABLE[operation]
    constraint = pulp.LpConstraint(expression,name=f"{product}-{operation}")
    prob += constraint
    constraints.append(constraint)
    #CAPACITY_REQUIREMENTS[('Cutting','Desks')] 

In [10]:
constraints

[0.4*Bookcases + 0.4*Chairs + 0.8*Desks + -100.0 = 0,
 3.0*Bookcases + 3.0*Chairs + 5.0*Desks + -650.0 = 0,
 1.1*Bookcases + 0.7*Chairs + 1.0*Desks + -140.0 = 0]

In [11]:
prob

ProductMix:
MAXIMIZE
25.0*Bookcases + 16.0*Chairs + 30.0*Desks + 0.0
SUBJECT TO
Bookcases_Cutting: 0.4 Bookcases + 0.4 Chairs + 0.8 Desks = 100

Bookcases_Finishing: 3 Bookcases + 3 Chairs + 5 Desks = 650

Bookcases_Storing: 1.1 Bookcases + 0.7 Chairs + Desks = 140

VARIABLES
Bookcases Continuous
Chairs Continuous
Desks Continuous

### Solve

In [12]:
prob.solve()

1

In [13]:
for product in product_dvs:
    print(product,product_dvs[product].value())

Desks 100.0
Chairs 37.5
Bookcases 12.5


In [14]:
prob.writeLP("product_mix.lp")

[Bookcases, Chairs, Desks]

In [15]:
prob.writeMPS("product_mix.mps")

[Bookcases, Chairs, Desks]